In [71]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
from torch import nn, optim
from torch.utils.data import random_split

In [8]:
!pip install openpyxl

     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     -------------------------------------- 250.0/250.0 kB 7.7 MB/s eta 0:00:00


### 데이터 읽어오기

In [78]:
def load_and_process_data(file_names):
    data = []
    labels = []
    for i, file_name in enumerate(file_names):
        df = pd.read_excel(file_name, header=None)
        for j in range(0, df.shape[0]):
            row1 = df.iloc[j, 1:7].astype(float)  # 데이터를 float으로 변환
            data.append(row1)
            labels.append(i)
    return np.array(data), np.array(labels)


class ExcelDataset(Dataset):
    def __init__(self, file_names):
        self.data, self.labels = load_and_process_data(file_names)
        print(type(self.data[0][0]))
        self.data = torch.tensor(self.data, dtype=torch.float32)
        self.labels = torch.tensor(self.labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [79]:
file_names=['0.xlsx', '1.xlsx', '2.xlsx']
dataset = ExcelDataset(file_names)

<class 'numpy.float64'>


## 학습 데이터로 만들기

In [80]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8)


## 신경망

In [81]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(6, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [82]:
net = Net()

## 학습

In [83]:
def train_and_test(net, train_dataloader, test_dataloader, epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    best_accuracy = 0
    for epoch in range(epochs):
        # 학습 단계
        net.train()
        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            inputs, labels = data
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # 테스트 단계
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_dataloader:
                inputs, labels = data
                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = correct / total
        print(f'Epoch {epoch + 1} loss: {running_loss / len(train_dataloader)}, accuracy: {correct / total}')
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(net.state_dict(), 'best_model.pth')
            
    print('Finished Training')

In [84]:
train_and_test(net, train_dataloader, test_dataloader, epochs=5000)

Epoch 1 loss: 4.810803751687746, accuracy: 0.3561643835616438
Epoch 2 loss: 1.1083125230428335, accuracy: 0.3561643835616438
Epoch 3 loss: 1.1049406915097624, accuracy: 0.3561643835616438
Epoch 4 loss: 1.1019258531364235, accuracy: 0.3561643835616438
Epoch 5 loss: 1.101130462981559, accuracy: 0.3561643835616438
Epoch 6 loss: 1.0991258878965635, accuracy: 0.3561643835616438
Epoch 7 loss: 1.100438701139914, accuracy: 0.3561643835616438
Epoch 8 loss: 1.100076697968148, accuracy: 0.3561643835616438
Epoch 9 loss: 1.0990149749291909, accuracy: 0.3561643835616438
Epoch 10 loss: 1.0989383040247738, accuracy: 0.3561643835616438
Epoch 11 loss: 1.0996250204137854, accuracy: 0.3561643835616438
Epoch 12 loss: 1.098769877408002, accuracy: 0.3561643835616438
Epoch 13 loss: 1.0987305029018506, accuracy: 0.3561643835616438
Epoch 14 loss: 1.0999420043584462, accuracy: 0.3561643835616438
Epoch 15 loss: 1.0997363973308254, accuracy: 0.3561643835616438
Epoch 16 loss: 1.0995498831207688, accuracy: 0.3561643

Epoch 145 loss: 1.098342380008182, accuracy: 0.2876712328767123
Epoch 146 loss: 1.099831848531156, accuracy: 0.2876712328767123
Epoch 147 loss: 1.0982400629971478, accuracy: 0.2876712328767123
Epoch 148 loss: 1.0996548388455365, accuracy: 0.2876712328767123
Epoch 149 loss: 1.0982516488513432, accuracy: 0.2876712328767123
Epoch 150 loss: 1.099680426958445, accuracy: 0.2876712328767123
Epoch 151 loss: 1.0981872242850226, accuracy: 0.2876712328767123
Epoch 152 loss: 1.0992417045541711, accuracy: 0.2876712328767123
Epoch 153 loss: 1.0980893019083384, accuracy: 0.2876712328767123
Epoch 154 loss: 1.098072367745477, accuracy: 0.2876712328767123
Epoch 155 loss: 1.0998125140731398, accuracy: 0.2876712328767123
Epoch 156 loss: 1.0995391961690542, accuracy: 0.2876712328767123
Epoch 157 loss: 1.0991553776973002, accuracy: 0.2876712328767123
Epoch 158 loss: 1.098861262604997, accuracy: 0.2876712328767123
Epoch 159 loss: 1.0983803981059306, accuracy: 0.2876712328767123
Epoch 160 loss: 1.099848934122

Epoch 285 loss: 1.0986483290388778, accuracy: 0.3561643835616438
Epoch 286 loss: 1.098753800263276, accuracy: 0.2876712328767123
Epoch 287 loss: 1.098715392318932, accuracy: 0.3561643835616438
Epoch 288 loss: 1.099978898022626, accuracy: 0.2876712328767123
Epoch 289 loss: 1.0989055214701473, accuracy: 0.3561643835616438
Epoch 290 loss: 1.0987597091777905, accuracy: 0.2876712328767123
Epoch 291 loss: 1.09882627628945, accuracy: 0.2876712328767123
Epoch 292 loss: 1.0984632582277865, accuracy: 0.2876712328767123
Epoch 293 loss: 1.098218308912741, accuracy: 0.2876712328767123
Epoch 294 loss: 1.0981986715986922, accuracy: 0.2876712328767123
Epoch 295 loss: 1.0998490404438328, accuracy: 0.2876712328767123
Epoch 296 loss: 1.0998378218831242, accuracy: 0.2876712328767123
Epoch 297 loss: 1.0992858925381221, accuracy: 0.2876712328767123
Epoch 298 loss: 1.0982938650492076, accuracy: 0.2876712328767123
Epoch 299 loss: 1.098136566780709, accuracy: 0.2876712328767123
Epoch 300 loss: 1.09975531294539

Epoch 418 loss: 1.0991782691027667, accuracy: 0.2876712328767123
Epoch 419 loss: 1.0989596747063302, accuracy: 0.2876712328767123
Epoch 420 loss: 1.0982353107349292, accuracy: 0.2876712328767123
Epoch 421 loss: 1.098922223658175, accuracy: 0.2876712328767123
Epoch 422 loss: 1.1001471248832908, accuracy: 0.2876712328767123
Epoch 423 loss: 1.0998740292884208, accuracy: 0.2876712328767123
Epoch 424 loss: 1.0995426822353054, accuracy: 0.2876712328767123
Epoch 425 loss: 1.0994664140649744, accuracy: 0.2876712328767123
Epoch 426 loss: 1.098471915399706, accuracy: 0.2876712328767123
Epoch 427 loss: 1.0982877950410586, accuracy: 0.2876712328767123
Epoch 428 loss: 1.09933539339014, accuracy: 0.2876712328767123
Epoch 429 loss: 1.0995830716313542, accuracy: 0.2876712328767123
Epoch 430 loss: 1.0991966144458667, accuracy: 0.2876712328767123
Epoch 431 loss: 1.0984139667974937, accuracy: 0.2876712328767123
Epoch 432 loss: 1.0989686347342826, accuracy: 0.2876712328767123
Epoch 433 loss: 1.09898857168

Epoch 557 loss: 1.0981170422322042, accuracy: 0.2876712328767123
Epoch 558 loss: 1.0982496222934208, accuracy: 0.2876712328767123
Epoch 559 loss: 1.099167114979512, accuracy: 0.2876712328767123
Epoch 560 loss: 1.0981369695147953, accuracy: 0.2876712328767123
Epoch 561 loss: 1.0997701400035136, accuracy: 0.2876712328767123
Epoch 562 loss: 1.0997714416400806, accuracy: 0.2876712328767123
Epoch 563 loss: 1.0994878749589663, accuracy: 0.2876712328767123
Epoch 564 loss: 1.0994571254060075, accuracy: 0.2876712328767123
Epoch 565 loss: 1.0991606357935313, accuracy: 0.2876712328767123
Epoch 566 loss: 1.0993969891522382, accuracy: 0.2876712328767123
Epoch 567 loss: 1.0990464977315955, accuracy: 0.2876712328767123
Epoch 568 loss: 1.099471936354766, accuracy: 0.2876712328767123
Epoch 569 loss: 1.0993505361917857, accuracy: 0.2876712328767123
Epoch 570 loss: 1.0986458256437972, accuracy: 0.2876712328767123
Epoch 571 loss: 1.0991520946090285, accuracy: 0.2876712328767123
Epoch 572 loss: 1.099334968

...

Epoch 4432 loss: 1.098829933114954, accuracy: 0.2876712328767123
Epoch 4433 loss: 1.09874778180509, accuracy: 0.2876712328767123
Epoch 4434 loss: 1.0996371765394468, accuracy: 0.2876712328767123
Epoch 4435 loss: 1.0983956407856297, accuracy: 0.2876712328767123
Epoch 4436 loss: 1.0995568230345443, accuracy: 0.2876712328767123
Epoch 4437 loss: 1.099322035505965, accuracy: 0.2876712328767123
Epoch 4438 loss: 1.0992277506235484, accuracy: 0.2876712328767123
Epoch 4439 loss: 1.0983943649240442, accuracy: 0.2876712328767123
Epoch 4440 loss: 1.0982908332670056, accuracy: 0.2876712328767123
Epoch 4441 loss: 1.099021534661989, accuracy: 0.2876712328767123
Epoch 4442 loss: 1.0981680348112777, accuracy: 0.2876712328767123
Epoch 4443 loss: 1.0980367596085008, accuracy: 0.2876712328767123
Epoch 4444 loss: 1.0981404749122825, accuracy: 0.2876712328767123
Epoch 4445 loss: 1.0978436470031738, accuracy: 0.2876712328767123
Epoch 4446 loss: 1.0998497299245886, accuracy: 0.2876712328767123
Epoch 4447 loss

Epoch 4569 loss: 1.0979319875304763, accuracy: 0.2876712328767123
Epoch 4570 loss: 1.0997043525850452, accuracy: 0.2876712328767123
Epoch 4571 loss: 1.0992779635094307, accuracy: 0.2876712328767123
Epoch 4572 loss: 1.0990192471323788, accuracy: 0.2876712328767123
Epoch 4573 loss: 1.0995980597831108, accuracy: 0.2876712328767123
Epoch 4574 loss: 1.0982697912164636, accuracy: 0.2876712328767123
Epoch 4575 loss: 1.0979872168721379, accuracy: 0.2876712328767123
Epoch 4576 loss: 1.0996580703838452, accuracy: 0.2876712328767123
Epoch 4577 loss: 1.099132454073107, accuracy: 0.2876712328767123
Epoch 4578 loss: 1.0989518101150926, accuracy: 0.2876712328767123
Epoch 4579 loss: 1.0996844124149632, accuracy: 0.2876712328767123
Epoch 4580 loss: 1.098835803366996, accuracy: 0.2876712328767123
Epoch 4581 loss: 1.0989045194677405, accuracy: 0.2876712328767123
Epoch 4582 loss: 1.098411276533797, accuracy: 0.2876712328767123
Epoch 4583 loss: 1.0988392056645573, accuracy: 0.2876712328767123
Epoch 4584 lo

Epoch 4705 loss: 1.0993605626595986, accuracy: 0.2876712328767123
Epoch 4706 loss: 1.0983726881645821, accuracy: 0.2876712328767123
Epoch 4707 loss: 1.0991750762269303, accuracy: 0.2876712328767123
Epoch 4708 loss: 1.098339464213397, accuracy: 0.2876712328767123
Epoch 4709 loss: 1.0992062317358482, accuracy: 0.2876712328767123
Epoch 4710 loss: 1.0994732347694602, accuracy: 0.2876712328767123
Epoch 4711 loss: 1.0990996844059713, accuracy: 0.2876712328767123
Epoch 4712 loss: 1.0989082149557166, accuracy: 0.2876712328767123
Epoch 4713 loss: 1.0994233086302474, accuracy: 0.2876712328767123
Epoch 4714 loss: 1.0987212593491014, accuracy: 0.2876712328767123
Epoch 4715 loss: 1.0990757523356258, accuracy: 0.2876712328767123
Epoch 4716 loss: 1.0989087336772196, accuracy: 0.2876712328767123
Epoch 4717 loss: 1.0987962258828652, accuracy: 0.3561643835616438
Epoch 4718 loss: 1.0985721704122182, accuracy: 0.2876712328767123
Epoch 4719 loss: 1.099744825749784, accuracy: 0.2876712328767123
Epoch 4720 l

Epoch 4846 loss: 1.0983404887689125, accuracy: 0.2876712328767123
Epoch 4847 loss: 1.0993416792637594, accuracy: 0.2876712328767123
Epoch 4848 loss: 1.0995242112391703, accuracy: 0.2876712328767123
Epoch 4849 loss: 1.0991103810233038, accuracy: 0.2876712328767123
Epoch 4850 loss: 1.098340392112732, accuracy: 0.2876712328767123
Epoch 4851 loss: 1.0981704802126497, accuracy: 0.2876712328767123
Epoch 4852 loss: 1.0994020120517627, accuracy: 0.2876712328767123
Epoch 4853 loss: 1.0980407805056185, accuracy: 0.2876712328767123
Epoch 4854 loss: 1.0981459520958565, accuracy: 0.2876712328767123
Epoch 4855 loss: 1.0979005066124168, accuracy: 0.2876712328767123
Epoch 4856 loss: 1.0995212696694039, accuracy: 0.2876712328767123
Epoch 4857 loss: 1.099147687087188, accuracy: 0.2876712328767123
Epoch 4858 loss: 1.0979915953971244, accuracy: 0.2876712328767123
Epoch 4859 loss: 1.0991941207164042, accuracy: 0.2876712328767123
Epoch 4860 loss: 1.0980232567400545, accuracy: 0.2876712328767123
Epoch 4861 l

Epoch 4979 loss: 1.0993193806828678, accuracy: 0.2876712328767123
Epoch 4980 loss: 1.0988907201870068, accuracy: 0.3561643835616438
Epoch 4981 loss: 1.0987750259605613, accuracy: 0.2876712328767123
Epoch 4982 loss: 1.0985230529630505, accuracy: 0.2876712328767123
Epoch 4983 loss: 1.098802995037388, accuracy: 0.2876712328767123
Epoch 4984 loss: 1.0994605921410225, accuracy: 0.2876712328767123
Epoch 4985 loss: 1.0984667249628015, accuracy: 0.2876712328767123
Epoch 4986 loss: 1.0993412217578373, accuracy: 0.2876712328767123
Epoch 4987 loss: 1.0990623718983419, accuracy: 0.2876712328767123
Epoch 4988 loss: 1.098367210981008, accuracy: 0.2876712328767123
Epoch 4989 loss: 1.099003769255973, accuracy: 0.2876712328767123
Epoch 4990 loss: 1.099573496225718, accuracy: 0.2876712328767123
Epoch 4991 loss: 1.0990464429597597, accuracy: 0.2876712328767123
Epoch 4992 loss: 1.0994874915561161, accuracy: 0.2876712328767123
Epoch 4993 loss: 1.0983801403561153, accuracy: 0.2876712328767123
Epoch 4994 los

## 예측

In [70]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
net.load_state_dict(torch.load('best_model.pth'))
net.eval()

with torch.no_grad():
    inputs = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8], dtype=torch.float32)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 0)
    print(f'Predicted class: {predicted.item()}')


Predicted class: 4


In [88]:
from sklearn.preprocessing import StandardScaler

def load_and_process_data(file_names):
    data = []
    labels = []
    scaler = StandardScaler()  # 데이터를 정규화
    for i, file_name in enumerate(file_names):
        df = pd.read_excel(file_name, header=None)
        df = df.dropna()  # 결측값 제거
        for j in range(0, df.shape[0]):
            row1 = df.iloc[j, 1:7].astype(float)
            data.append(row1)
            labels.append(i)
    data = scaler.fit_transform(data)  # 데이터 정규화
    return np.array(data), np.array(labels)

class ExcelDataset(Dataset):
    def __init__(self, file_names):
        self.data, self.labels = load_and_process_data(file_names)
        print(type(self.data[0][0]))
        self.data = torch.tensor(self.data, dtype=torch.float32)
        self.labels = torch.tensor(self.labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

file_names=['0.xlsx', '1.xlsx', '2.xlsx']
dataset = ExcelDataset(file_names)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(6, 16)
        nn.init.xavier_uniform_(self.fc1.weight)  # 가중치 초기화
        self.fc2 = nn.Linear(16, 16)
        nn.init.xavier_uniform_(self.fc2.weight)  # 가중치 초기화
        self.fc3 = nn.Linear(16, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

def train_and_test(net, train_dataloader, test_dataloader, epochs=500):  # 에폭 수 줄임
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001)  # 학습률 줄임
    best_accuracy = 0
    for epoch in range(epochs):
        net.train()
        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            inputs, labels = data
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_dataloader:
                inputs, labels = data
                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f'Epoch {epoch + 1} loss: {running_loss / len(train_dataloader)}, accuracy: {correct / total}')
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(net.state_dict(), 'best_model.pth')
            
    print('Finished Training')

train_and_test(net, train_dataloader, test_dataloader, epochs=500)


<class 'numpy.float64'>
Epoch 1 loss: 1.146785855293274, accuracy: 0.34782608695652173
Epoch 2 loss: 1.1281412939230602, accuracy: 0.34782608695652173
Epoch 3 loss: 1.119953175385793, accuracy: 0.391304347826087
Epoch 4 loss: 1.0978326400121052, accuracy: 0.43478260869565216
Epoch 5 loss: 1.0696574499209721, accuracy: 0.4782608695652174
Epoch 6 loss: 1.0664108196894329, accuracy: 0.4782608695652174
Epoch 7 loss: 1.062471608320872, accuracy: 0.4782608695652174
Epoch 8 loss: 1.0495869666337967, accuracy: 0.4782608695652174
Epoch 9 loss: 1.0535227904717128, accuracy: 0.4782608695652174
Epoch 10 loss: 1.032950982451439, accuracy: 0.5217391304347826
Epoch 11 loss: 1.0253314971923828, accuracy: 0.5217391304347826
Epoch 12 loss: 1.014140377442042, accuracy: 0.5217391304347826
Epoch 13 loss: 1.011789694428444, accuracy: 0.5217391304347826
Epoch 14 loss: 1.0014522274335225, accuracy: 0.5652173913043478
Epoch 15 loss: 0.9858882824579874, accuracy: 0.5652173913043478
Epoch 16 loss: 0.989259769519

Epoch 165 loss: 0.33853716651598614, accuracy: 1.0
Epoch 166 loss: 0.3339039906859398, accuracy: 1.0
Epoch 167 loss: 0.335799985875686, accuracy: 1.0
Epoch 168 loss: 0.325185580799977, accuracy: 1.0
Epoch 169 loss: 0.3181381858885288, accuracy: 1.0
Epoch 170 loss: 0.35755106806755066, accuracy: 1.0
Epoch 171 loss: 0.36083802084128064, accuracy: 1.0
Epoch 172 loss: 0.31709735716382664, accuracy: 1.0
Epoch 173 loss: 0.3126759963730971, accuracy: 1.0
Epoch 174 loss: 0.31946641703446704, accuracy: 1.0
Epoch 175 loss: 0.32070036853353184, accuracy: 1.0
Epoch 176 loss: 0.3042573469380538, accuracy: 1.0
Epoch 177 loss: 0.3068893315891425, accuracy: 1.0
Epoch 178 loss: 0.29993889791270095, accuracy: 1.0
Epoch 179 loss: 0.304829736550649, accuracy: 1.0
Epoch 180 loss: 0.30047477160890895, accuracy: 1.0
Epoch 181 loss: 0.30948153262337047, accuracy: 1.0
Epoch 182 loss: 0.29838524758815765, accuracy: 1.0
Epoch 183 loss: 0.2939522551993529, accuracy: 1.0
Epoch 184 loss: 0.2907621239622434, accurac

Epoch 332 loss: 0.17027066182345152, accuracy: 1.0
Epoch 333 loss: 0.18150322263439497, accuracy: 1.0
Epoch 334 loss: 0.1697586647545298, accuracy: 1.0
Epoch 335 loss: 0.17911106689522663, accuracy: 1.0
Epoch 336 loss: 0.16893534424404302, accuracy: 1.0
Epoch 337 loss: 0.17380733756969372, accuracy: 1.0
Epoch 338 loss: 0.17549589090049267, accuracy: 1.0
Epoch 339 loss: 0.1703992299735546, accuracy: 1.0
Epoch 340 loss: 0.168500746289889, accuracy: 1.0
Epoch 341 loss: 0.17196234843383232, accuracy: 1.0
Epoch 342 loss: 0.1999444772178928, accuracy: 1.0
Epoch 343 loss: 0.16526415618136525, accuracy: 1.0
Epoch 344 loss: 0.16414760906870166, accuracy: 1.0
Epoch 345 loss: 0.16620879403005043, accuracy: 1.0
Epoch 346 loss: 0.1652014795690775, accuracy: 1.0
Epoch 347 loss: 0.16296684741973877, accuracy: 1.0
Epoch 348 loss: 0.1646722499281168, accuracy: 1.0
Epoch 349 loss: 0.1666708036015431, accuracy: 1.0
Epoch 350 loss: 0.16292559960857034, accuracy: 1.0
Epoch 351 loss: 0.16422687005251646, ac

Epoch 496 loss: 0.13143322704127058, accuracy: 1.0
Epoch 497 loss: 0.13789080828428268, accuracy: 1.0
Epoch 498 loss: 0.14297715543458858, accuracy: 1.0
Epoch 499 loss: 0.1359681605050961, accuracy: 1.0
Epoch 500 loss: 0.13583379114667574, accuracy: 1.0
Finished Training
